<a href="https://colab.research.google.com/github/prateekjoshi565/textrank_text_summarization/blob/master/TestRank_Text_Summarization.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import nltk
from nltk.tokenize import sent_tokenize
nltk.download('punkt') # one time execution
import re

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\GeorgiVankov\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
# Read the CSV file
import io
df = pd.read_csv("tennis_articles_v4.csv", header=None, dtype=object)

In [5]:
df.head()

,0,1,2
0,article_id,article_text,source
1,1,Maria Sharapova has basically no friends as te...,https://www.tennisworldusa.org/tennis/news/Mar...
2,2,"BASEL, Switzerland (AP), Roger Federer advance...",http://www.tennis.com/pro-game/2018/10/copil-s...
3,3,Roger Federer has revealed that organisers of ...,https://scroll.in/field/899938/tennis-roger-fe...
4,4,Kei Nishikori will try to end his long losing ...,http://www.tennis.com/pro-game/2018/10/nishiko...


In [6]:
# split the the text in the articles into sentences
sentences = []
for s in df[1]:
  sentences.append(sent_tokenize(s))  
print(sentences)

[['article_text'], ['Maria Sharapova has basically no friends as tennis players on the WTA Tour.', "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.", 'I think everyone knows this is my job here.', "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.", "I'm a pretty competitive girl.", "I say my hellos, but I'm not sending any players flowers as well.", "Uhm, I'm not really friendly or close to many players.", "I have not a lot of friends away from the courts.'", 'When she said she is not really close to a lot of players, is that something strategic that she is doing?', "Is it different on the men's tour than the women's tour?", "'No, no

In [7]:
# flatten the list
sentences = [y for x in sentences for y in x]
print(sentences)

['article_text', 'Maria Sharapova has basically no friends as tennis players on the WTA Tour.', "The Russian player has no problems in openly speaking about it and in a recent interview she said: 'I don't really hide any feelings too much.", 'I think everyone knows this is my job here.', "When I'm on the courts or when I'm on the court playing, I'm a competitor and I want to beat every single person whether they're in the locker room or across the net.So I'm not the one to strike up a conversation about the weather and know that in the next few minutes I have to go and try to win a tennis match.", "I'm a pretty competitive girl.", "I say my hellos, but I'm not sending any players flowers as well.", "Uhm, I'm not really friendly or close to many players.", "I have not a lot of friends away from the courts.'", 'When she said she is not really close to a lot of players, is that something strategic that she is doing?', "Is it different on the men's tour than the women's tour?", "'No, not a

In [8]:
# remove punctuations, numbers and special characters
clean_sentences = pd.Series(sentences).str.replace("[^a-zA-Z]", " ")
print(clean_sentences)

# make alphabets lowercase
clean_sentences = [s.lower() for s in clean_sentences]
print(clean_sentences)

0                                           article text
1      Maria Sharapova has basically no friends as te...
2      The Russian player has no problems in openly s...
3            I think everyone knows this is my job here 
4      When I m on the courts or when I m on the cour...
                             ...                        
115    It makes me incredibly happy to win my home to...
116     I do not know if it s maybe my last title  so...
117     Maybe I should celebrate as if it were my las...
118     There are very touching moments  seeing the b...
119    Because it was not always easy in the last wee...
Length: 120, dtype: object
['article text', 'maria sharapova has basically no friends as tennis players on the wta tour ', 'the russian player has no problems in openly speaking about it and in a recent interview she said   i don t really hide any feelings too much ', 'i think everyone knows this is my job here ', 'when i m on the courts or when i m on the court playing 

c:\Users\GeorgiVankov\Desktop\AI\AI-Project\venv\lib\site-packages\ipykernel_launcher.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  


In [9]:
nltk.download('stopwords')# one time execution

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\GeorgiVankov\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [10]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [11]:
# function to remove stopwords
def remove_stopwords(sen):
  sen_new = " ".join([i for i in sen if i not in stop_words])
  return sen_new

In [12]:
# remove stopwords from the sentences
clean_sentences = [remove_stopwords(r.split()) for r in clean_sentences]
print(clean_sentences)

['article text', 'maria sharapova basically friends tennis players wta tour', 'russian player problems openly speaking recent interview said really hide feelings much', 'think everyone knows job', 'courts court playing competitor want beat every single person whether locker room across net one strike conversation weather know next minutes go try win tennis match', 'pretty competitive girl', 'say hellos sending players flowers well', 'uhm really friendly close many players', 'lot friends away courts', 'said really close lot players something strategic', 'different men tour women tour', '', 'think sport mean friends everyone categorized tennis player going get along tennis players', 'think every person different interests', 'friends completely different jobs interests met different parts life', 'think everyone thinks tennis players greatest friends', 'ultimately tennis small part', 'many things interested', 'basel switzerland ap roger federer advanced th swiss indoors final career beatin

In [13]:
# download pretrained GloVe word embeddings
!wget http://nlp.stanford.edu/data/glove.6B.zip

'wget' is not recognized as an internal or external command,
operable program or batch file.


In [27]:
from gensim.models import Word2Vec
import multiprocessing
cores = multiprocessing.cpu_count()
w2v_model = Word2Vec(vector_size=10,
                    min_count=1)
w2v_model.build_vocab(sentences)

total_examples = w2v_model.corpus_count
w2v_model.train(sentences, total_examples=total_examples, epochs=w2v_model.epochs)
sentence_embeddings=[[sum(w2v_model.wv[word]) for word in words] for words in sentences]
max_len=max([len(tokens) for tokens in sentences])
sentence_embeddings=[np.pad(embedding,(0,max_len-len(embedding)),'constant') for embedding in sentence_embeddings]
print(sentence_embeddings)
        

[array([0.3062713 , 0.65793763, 0.57457385, 0.45435631, 0.49365207,
       0.52297722, 0.24301048, 0.08074636, 0.57457385, 0.24301048,
       0.04597994, 0.57457385, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.    

In [22]:

from scipy import spatial
similarity_matrix = np.zeros([len(clean_sentences), len(clean_sentences)])
for i,row_embedding in enumerate(sentence_embeddings):
    for j,column_embedding in enumerate(sentence_embeddings):
        similarity_matrix[i][j]=1-spatial.distance.cosine(row_embedding,column_embedding)
print(similarity_matrix)

[[1.         0.30344811 0.25601306 ... 0.33367941 0.26869844 0.38704226]
 [0.30344811 1.         0.63456069 ... 0.72300466 0.69492265 0.72325339]
 [0.25601306 0.63456069 1.         ... 0.49959786 0.80545575 0.49010707]
 ...
 [0.33367941 0.72300466 0.49959786 ... 1.         0.59144101 0.81059671]
 [0.26869844 0.69492265 0.80545575 ... 0.59144101 1.         0.57058211]
 [0.38704226 0.72325339 0.49010707 ... 0.81059671 0.57058211 1.        ]]


In [30]:
import networkx as nx
nx_graph = nx.from_numpy_array(similarity_matrix)
scores = nx.pagerank_numpy(nx_graph)
print(scores)

{0: 0.004514018528371271, 1: 0.00871904570449734, 2: 0.008606813675620507, 3: 0.007649278940131938, 4: 0.006524941743727229, 5: 0.006672549824919781, 6: 0.008401644310452575, 7: 0.0082376018228639, 8: 0.007914847181113106, 9: 0.009085955461528293, 10: 0.008122863034886426, 11: 0.005423750943386333, 12: 0.007768398211831734, 13: 0.00769468782028245, 14: 0.008830635262733458, 15: 0.008786986516438274, 16: 0.008478498349103148, 17: 0.008614141305041253, 18: 0.008149846583528967, 19: 0.00878041480928524, 20: 0.008955062852325241, 21: 0.009064545817764997, 22: 0.008739904288006245, 23: 0.0083559879001806, 24: 0.008995808186781678, 25: 0.007286570934456532, 26: 0.007258915564056328, 27: 0.008357929215880768, 28: 0.008830268640888116, 29: 0.009068716530990981, 30: 0.008324552972193228, 31: 0.007238614365392808, 32: 0.008284135646084625, 33: 0.008580318391750382, 34: 0.008741741019362127, 35: 0.007325453301609203, 36: 0.008070434341680761, 37: 0.008858174568576722, 38: 0.008159391876643513, 39

c:\Users\GeorgiVankov\Desktop\AI\AI-Project\venv\lib\site-packages\ipykernel_launcher.py:3: DeprecationWarning: networkx.pagerank_numpy is deprecated and will be removed in NetworkX 3.0, use networkx.pagerank instead.
  This is separate from the ipykernel package so we can avoid doing imports until


The next step is to find similarities among the sentences. We will use cosine similarity to find similarity between a pair of sentences. Let's create an empty similarity matrix for this task and populate it with cosine similarities of the sentences.

In [35]:
ranked_sentences = sorted(((scores[i],s) for i,s in enumerate(sentences)), reverse=True)
print(ranked_sentences)

[(0.009143694900239988, "Federer's projected route to the Paris final could also lead to matches against Kevin Anderson and Novak Djokovic."), (0.00913820106701193, 'Federer, 37, first broke through on tour over two decades ago and he has since gone on to enjoy a glittering career.'), (0.009085955461528293, 'When she said she is not really close to a lot of players, is that something strategic that she is doing?'), (0.009068716530990981, 'Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago.'), (0.009064545817764997, "He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand."), (0.009048311442545844, 'Anderson has a shot at a fifth career title and second of the year after winning in New York in February.'), (0.009036321923237507, 'But she claims the mentality of professional tennis players slowly began to change after the new millennium.'), (0.0090258259259

In [36]:
# Specify number of sentences to form the summary
sn = 10

# Generate summary
for i in range(sn):
  print(ranked_sentences[i][1])

Federer's projected route to the Paris final could also lead to matches against Kevin Anderson and Novak Djokovic.
Federer, 37, first broke through on tour over two decades ago and he has since gone on to enjoy a glittering career.
When she said she is not really close to a lot of players, is that something strategic that she is doing?
Federer had an easier time than in his only previous match against Medvedev, a three-setter at Shanghai two weeks ago.
He then dropped his serve to love, and let another match point slip in Medvedev's next service game by netting a backhand.
Anderson has a shot at a fifth career title and second of the year after winning in New York in February.
But she claims the mentality of professional tennis players slowly began to change after the new millennium.
"We also had the impression that at this stage it might be better to play matches than to train.
"There are very touching moments: seeing the ball children, the standing ovations, all the familiar faces in